In [1]:
import itertools
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import random
from scipy import linalg
from scipy.optimize import minimize
from scipy.stats import multivariate_normal
from sklearn.cluster import KMeans
from sklearn.cluster import spectral_clustering
from sklearn.metrics.cluster import normalized_mutual_info_score
from tqdm import tqdm
from web_data import pre_data, get_laplacian

In [2]:
result = pre_data()

# Adjacency Matrix
adj = result[0]

# Import Feature
X = result[1]
X = np.transpose(X)

# Label
label = result[2]

# Delete Two-way Reference
adj = adj.toarray()
row, col = np.diag_indices_from(adj)
adj[row, col] = 0

# Laplacian Matrix
A, D, L, dist = get_laplacian(adj, label = label, draw_plot = False)

Dataset has 195 nodes, 304 edges, 1703 features.


In [3]:
# Outlier Detection
outlier_count = []
for i in range(dist.shape[0]):
    outlier_count.append(np.sum(dist[i,:] == 1000))

t = pd.Series(outlier_count)
t = t.value_counts()
print('-'*30 + 'Number of Isolation Points' + '-'*30)
print(t.sort_index())

outlier = []
for i in range(dist.shape[0]):
    if np.sum(dist[i,:] == 1000) > 12:
        outlier.append(i)
print('-'*30 + 'Outlier' + '-'*30)
print(len(outlier))
print(outlier)

------------------------------Number of Isolation Points------------------------------
12     183
193     12
dtype: int64
------------------------------Outlier------------------------------
12
[11, 15, 16, 71, 73, 74, 124, 125, 130, 131, 141, 186]


In [4]:
# Outlier Deletion
outlier = np.array(outlier)

def delete_outlier(X, adj, A, D):

    # Delete X
    X = X.toarray()
    X = np.delete(X, outlier, axis = 1)

    # Delete adj
    adjacency_matrix = adj
    adjacency_matrix = np.delete(adjacency_matrix, outlier, axis = 1)
    adjacency_matrix = np.delete(adjacency_matrix, outlier, axis = 0)

    # Delete A
    A = np.delete(A, outlier, axis = 1)
    A = np.delete(A, outlier, axis = 0)

    # Delete D
    D = np.delete(D, outlier, axis = 1)
    D = np.delete(D, outlier, axis = 0)

    # Delete L
    L = D - A
    
    return X, adjacency_matrix, A, D, L

X, adjacency_matrix, A, D, L = delete_outlier(X, adj, A, D)

In [5]:
# Check Shape of X, adj, A, D, L
print(X.shape)
print(adj.shape)
print(A.shape)
print(D.shape)
print(L.shape)

(1703, 183)
(195, 195)
(183, 183)
(183, 183)
(183, 183)


In [6]:
# Feature Selection
feature = np.transpose(X.copy())
agreement = [0] * feature.shape[1]
adjacency_upper = np.triu(adjacency_matrix)
rows, cols = np.where((adjacency_upper == 1) | (adjacency_upper == 2))  # 2: Two-way Reference (?)
edges = list(zip(rows.tolist(), cols.tolist()))
for i in range(len(edges)):
    flow = edges[i]
    p1 = flow[0]
    p2 = flow[1]
    for j in range(feature.shape[1]):
        if feature[p1, j] == feature[p2, j]:
            agreement[j] += 1

agreement = np.array(agreement)
print(np.quantile(agreement, 0.95))
ind = np.where(agreement > np.quantile(agreement, 0.95))[0]
print(X.shape)

277.0
(1703, 183)


In [7]:
# Use Normalized Laplacian Matrix
L = np.identity(D.shape[0]) - np.dot(np.dot(np.linalg.inv(D ** 0.5), A), np.linalg.inv(D ** 0.5))

In [8]:
# True Label
label = np.delete(label, outlier, axis = 0)
classnames, label = np.unique(label, return_inverse = True)

In [ ]:
# ---------- SSC Clustering with Graph Information ---------- #

In [9]:
# Define Needed Functions
def f_1(Z):                    # First Term
    return (1/2) * (np.linalg.norm(X - np.dot(X, Z))) ** 2

def f_2(Z, lambda_1):          # Second Term
    return lambda_1 * np.linalg.norm(Z, ord = 1)

def f_3(Z, lambda_2):          # Third Term
    return lambda_2 * np.trace(np.dot(np.transpose(Z), np.dot(L, Z)))

def f(Z, lambda_1, lambda_2):  # Complete
    return f_1(Z) + f_2(Z, lambda_1 = lambda_1) + f_3(Z, lambda_2 = lambda_2)

def G(Z, lambda_2):
    return lambda_2 * np.dot(L, Z) - np.dot(np.transpose(X), X - np.dot(X, Z))

def C(Z, lambda_2, mu):
    return Z - G(Z, lambda_2 = lambda_2) / mu

In [10]:
# SSC Clustering
m = L.shape[0]
lambda_3 = 1

def SSC(lambda_1, lambda_2, X):

    # Initialization
    Z = np.dot(np.linalg.inv(np.dot(np.transpose(X), X) + lambda_2 * L + lambda_3 * np.identity(m)), np.dot(np.transpose(X), X))
    Z = Z - np.diag(np.diag(Z))    
    mu = 1.02 * np.linalg.norm(np.dot(np.transpose(X), X) + lambda_2 * L, ord = 2)
    
    # Iteration
    itr = 0
    while True:
        Z0 = Z # Store the initial value
        Z = np.maximum(C(Z, mu = mu, lambda_2 = lambda_2) - lambda_2/mu, 0) + np.minimum(C(Z, mu = mu, lambda_2 = lambda_2) + lambda_2/mu, 0)
        Z = Z - np.diag(np.diag(Z))
        itr += 1
        if np.linalg.norm(Z - Z0) / np.linalg.norm(Z0) <= 1e-2 and abs(f(Z, lambda_1 = lambda_1, lambda_2 = lambda_2) - f(Z0, lambda_1 = lambda_1, lambda_2 = lambda_2)) <= 1e-2 :
            break
        if itr > 1e+6:
            print('-'*10 + 'lambda_1 = ' + str(lambda_1) + ', lambda_2 = ' + str(lambda_2) + ', Warning: Not Converge!' + '-'*10)
            break
    
    # Return Symmetric Matrix W
    W = np.abs(Z + np.transpose(Z))
    return W

In [11]:
# Adjust the Parameters
def expandgrid(*itrs):
   product = list(itertools.product(*itrs))
   return {'Var{}'.format(i+1):[x[i] for x in product] for i in range(len(itrs))}

a = [0.01, 0.1, 1, 10]
b = [0.01, 0.1, 1, 10]
candidate = pd.DataFrame(expandgrid(a, b))
candidate.columns = ['lambda_1', 'lambda_2']

W = {}
cluster_result = {}
score = []
accuracy = []
label_count = pd.Series(label)
label_ind = label_count.value_counts().index

for i in tqdm(range(candidate.shape[0])):

    lambda_1 = candidate.iloc[i, 0]
    lambda_2 = candidate.iloc[i, 1]
    ssc_result = SSC(lambda_1, lambda_2, X)
    W[i] = ssc_result
    ClusterResult = spectral_clustering(ssc_result, n_clusters = 5)
    cluster_result[i] = ClusterResult

    # NMI
    score.append(normalized_mutual_info_score(label, ClusterResult))

    # Accuracy
    result_count = pd.Series(ClusterResult)
    result_ind = result_count.value_counts().index
    true_number = 0
    for j in range(len(label_ind)):
        true_label = label_ind[j]
        true_number += np.sum(ClusterResult[label == true_label] == result_ind[j])
    accuracy.append(true_number / len(label))

score_result = pd.concat([candidate, pd.DataFrame(score), pd.DataFrame(accuracy)], axis = 1)
score_result.columns = ['lambda_1', 'lambda_2', 'NMI', 'Accuracy']
score_result

 19%|█▉        | 3/16 [04:42<18:55, 87.31s/it] C:\Users\THINK\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\manifold\_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
 44%|████▍     | 7/16 [10:48<14:17, 95.25s/it] C:\Users\THINK\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\manifold\_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
 69%|██████▉   | 11/16 [15:34<06:41, 80.26s/it]C:\Users\THINK\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\manifold\_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
 94%|█████████▍| 15/16 [21:06<01:26, 86.25s

,lambda_1,lambda_2,NMI,Accuracy
0,0.01,0.01,0.189798,0.377049
1,0.01,0.10,0.299447,0.568306
2,0.01,1.00,0.097372,0.469945
3,0.01,10.00,0.423306,0.426230
4,0.10,0.01,0.171664,0.371585
5,0.10,0.10,0.299447,0.568306
6,0.10,1.00,0.097372,0.469945
7,0.10,10.00,0.423306,0.426230
8,1.00,0.01,0.174836,0.371585
9,1.00,0.10,0.295341,0.573770


In [12]:
score_result.iloc[np.where(score_result['NMI'] == max(score_result['NMI']))[0], :]

,lambda_1,lambda_2,NMI,Accuracy
3,0.01,10.0,0.423306,0.42623
7,0.10,10.0,0.423306,0.42623
11,1.00,10.0,0.423306,0.42623
15,10.00,10.0,0.423306,0.42623


In [13]:
score_result.iloc[np.where(score_result['Accuracy'] == max(score_result['Accuracy']))[0], :]

,lambda_1,lambda_2,NMI,Accuracy
13,10.0,0.1,0.308601,0.584699


In [ ]:
# ---------- Standard SSC ---------- #

In [14]:
# Standard SSC
class ssc_model(object):
    def __init__(self, X, affine=False, alpha1=800, alpha2=None, thr=0.0002, maxIter=200):
        self.alpha1 = alpha1 
        if not alpha2:
            self.alpha2 = alpha1
        else:
            self.alpha2 = alpha2
        
        self.X = X
        self.affine = affine    
        self.thr = thr
        self.maxIter = maxIter
        self.N = X.shape[1]   # number of samples
        
        self.T = np.dot(self.X.T,self.X)
        T1 = np.abs(self.T - np.diag(np.diag(self.T)))
        self.lambda1 = np.min(np.max(T1,axis=1))
        self.mu1 = self.alpha1/self.lambda1
        self.mu2 = self.alpha2 
        self.I = np.eye(self.N,dtype=np.float32)
        self.ones = np.ones((self.N,self.N),dtype=np.float32)
        self.vec1N = np.ones((1,self.N),dtype = np.float32)
        self.err =[]
        
    def computeCmat(self):
        if not self.affine:
            A = np.linalg.inv(self.mu1*self.T + self.mu2*self.I)
            C1 = np.zeros((self.N,self.N),dtype=np.float32)
            Lambda2 = np.zeros((self.N,self.N),dtype=np.float32)
            err = 10*self.thr
            iter1 = 1
            while (err>self.thr)and(iter1<self.maxIter):
                # update Z
                Z = np.dot(A,self.mu1*self.T + self.mu2*(C1 - Lambda2/self.mu2))
                Z = Z - np.diag(np.diag(Z))
                # update C
                tmp_val = np.abs(Z + Lambda2/self.mu2) - (self.ones/self.mu2)
                C2 = np.maximum(0,tmp_val)*np.sign(Z + Lambda2/self.mu2)
                C2 = C2 - np.diag(np.diag(C2))
                # update lagrangian multipliers
                Lambda2 = Lambda2 + self.mu2*(Z-C2)
                # compute errors
                tmp_val = np.abs(Z - C2)
                err = np.max(tmp_val.reshape(-1,1))
                C1 = C2
                iter1 = iter1 + 1
        else:
            A = np.linalg.inv(self.mu1*self.T + self.mu2*self.I+ self.mu2*self.ones)
            C1 = np.zeros((self.N,self.N),dtype=np.float32)
            Lambda2 = np.zeros((self.N,self.N),dtype=np.float32)
            Lambda3 = np.zeros((1,self.N),dtype=np.float32)
            err1 = 10*self.thr
            err3 = 10*self.thr
            iter1 = 1
            while ((err1>self.thr)or(err3>self.thr))and(iter1<self.maxIter):
                # update Z
                tmp_val = self.mu1*self.T + self.mu2*(C1-Lambda2/self.mu2) + self.mu2*np.dot(self.vec1N.T,(self.vec1N - Lambda3/self.mu2))
                Z = np.dot(A,tmp_val)
                Z = Z - np.diag(np.diag(Z))
                # update C
                tmp_val = np.abs(Z + Lambda2/self.mu2) - (self.ones/self.mu2)
                C2 = np.maximum(0,tmp_val)*np.sign(Z + Lambda2/self.mu2)
                C2 = C2 - np.diag(np.diag(C2))
                # update lagrangian multipliers
                Lambda2 = Lambda2 + self.mu2*(Z-C2)
                Lambda3 = Lambda3 + self.mu2*(np.dot(self.vec1N,Z) - self.vec1N)
                # compute errors
                tmp_val = np.abs(Z - C2)
                err1 = np.max(tmp_val.reshape(-1,1))
                tmp_val = np.abs(np.dot(self.vec1N,Z) - self.vec1N)
                err3 = np.max(tmp_val.reshape(-1,1))
                
                C1 = C2
                iter1 = iter1 + 1
        return C2

In [15]:
# Adjust the Parameters
def expandgrid(*itrs):
   product = list(itertools.product(*itrs))
   return {'Var{}'.format(i+1):[x[i] for x in product] for i in range(len(itrs))}

a = [0.01, 0.1, 1, 10]
b = [0.01, 0.1, 1, 10]
candidate = pd.DataFrame(expandgrid(a, b))
candidate.columns = ['alpha1', 'alpha2']

score = []
accuracy = []
label_count = pd.Series(label)
label_ind = label_count.value_counts().index

for i in tqdm(range(candidate.shape[0])):

    alpha1 = candidate.iloc[i, 0]
    alpha2 = candidate.iloc[i, 1]
    ss = ssc_model(X, alpha1, alpha2)
    Z = ss.computeCmat()
    W = np.abs(Z + np.transpose(Z))
    ClusterResult = spectral_clustering(W, n_clusters = 5)

    # NMI
    score.append(normalized_mutual_info_score(label, ClusterResult))

    # Accuracy
    result_count = pd.Series(ClusterResult)
    result_ind = result_count.value_counts().index
    true_number = 0
    for j in range(len(label_ind)):
        true_label = label_ind[j]
        true_number += np.sum(ClusterResult[label == true_label] == result_ind[j])
    accuracy.append(true_number / len(label))

score_result = pd.concat([candidate, pd.DataFrame(score), pd.DataFrame(accuracy)], axis = 1)
score_result.columns = ['alpha1', 'alpha2', 'NMI', 'Accuracy']
score_result

  0%|          | 0/16 [00:00<?, ?it/s]C:\Users\THINK\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\manifold\_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
 25%|██▌       | 4/16 [00:01<00:04,  2.75it/s]C:\Users\THINK\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\manifold\_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
 50%|█████     | 8/16 [00:02<00:03,  2.53it/s]C:\Users\THINK\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\manifold\_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
 75%|███████▌  | 12/16 [00:04<00:01,  2.49it/s]C:\User

,alpha1,alpha2,NMI,Accuracy
0,0.01,0.01,0.042980,0.415301
1,0.01,0.10,0.040705,0.415301
2,0.01,1.00,0.130414,0.426230
3,0.01,10.00,0.097372,0.469945
4,0.10,0.01,0.032525,0.344262
5,0.10,0.10,0.070073,0.371585
6,0.10,1.00,0.130414,0.426230
7,0.10,10.00,0.097372,0.469945
8,1.00,0.01,0.027312,0.344262
9,1.00,0.10,0.068168,0.360656


In [16]:
score_result.iloc[np.where(score_result['NMI'] == max(score_result['NMI']))[0], :]

,alpha1,alpha2,NMI,Accuracy
10,1.0,1.0,0.138041,0.442623
14,10.0,1.0,0.138041,0.442623


In [17]:
score_result.iloc[np.where(score_result['Accuracy'] == max(score_result['Accuracy']))[0], :]

,alpha1,alpha2,NMI,Accuracy
3,0.01,10.0,0.097372,0.469945
7,0.10,10.0,0.097372,0.469945
11,1.00,10.0,0.097372,0.469945
15,10.00,10.0,0.097372,0.469945


In [ ]:
# ---------- K-Means ---------- #

In [18]:
# K-means
np.random.seed(42)
kmeansResult = KMeans(n_clusters = 5, random_state = 10).fit(np.transpose(X)).labels_

In [19]:
# NMI
normalized_mutual_info_score(label, kmeansResult)

0.3271984961914506

In [20]:
# Accuracy
label_count = pd.Series(label)
label_ind = label_count.value_counts().index

result_count = pd.Series(kmeansResult)
result_ind = result_count.value_counts().index

true_number = 0
for j in range(len(label_ind)):
    true_label = label_ind[j]
    true_number += np.sum(kmeansResult[label == true_label] == result_ind[j])
true_number / len(label)

0.4098360655737705

In [ ]:
# ---------- Balanced K-Means ---------- #

In [21]:
# Balanced K-Means
def objective(X, H, F, gamma):
    one = np.ones((np.shape(F)[0], 1))
    FOne = np.dot(np.transpose(F), one)
    return (np.linalg.norm(X - np.dot(H, np.transpose(F)))) ** 2 + gamma * np.trace(np.dot(FOne, np.transpose(FOne)))

def balancedKMeans(X, n_clusters, gamma):

    d = np.shape(X)[0]
    n = np.shape(X)[1]
    k = n_clusters

    # Initialization
    np.random.seed(42)
    F = np.zeros((n, k))
    for i in range(n):
        position = np.random.randint(0, k) 
        F[i, position] = 1

    # Iteration
    while True:
        F0 = F.copy()
        H = np.dot(np.dot(X, F), np.linalg.inv(np.dot(np.transpose(F), F)))
        for i in range(n):
            position = np.where(F[i, ] == 1)[0][0]
            benchmark = objective(X, H, F, gamma)
            F[i, position] = 0
            F_copy = F.copy()
            for j in range(k):
                F_copy[i, j] = 1
                value = objective(X, H, F_copy, gamma)
                if value < benchmark:
                    position = j
                    benchmark = value
                F_copy[i, j] = 0
            F[i, position] = 1
        if np.linalg.norm(F - F0) / np.linalg.norm(F0) <= 1e-2:
            break

    return F

In [22]:
# Adjust the Parameters
def expandgrid(*itrs):
   product = list(itertools.product(*itrs))
   return {'Var{}'.format(i+1):[x[i] for x in product] for i in range(len(itrs))}

a = [0.01, 0.05, 0.1, 0.5, 1, 5, 10]
candidate = pd.DataFrame(expandgrid(a))
candidate.columns = ['gamma']

score = []
accuracy = []
label_count = pd.Series(label)
label_ind = label_count.value_counts().index

for i in tqdm(range(candidate.shape[0])):

    gamma = a[i]
    mingzi = balancedKMeans(np.array(X), 5, gamma)
    ClusterResult = np.zeros(np.shape(mingzi)[0])
    for j in range(np.shape(mingzi)[0]):
        for k in range(np.shape(mingzi)[1]):
            if mingzi[j, k] == 1:
                ClusterResult[j] = k

    # NMI
    score.append(normalized_mutual_info_score(label, ClusterResult))

    # Accuracy
    result_count = pd.Series(ClusterResult)
    result_ind = result_count.value_counts().index
    true_number = 0
    for l in range(len(label_ind)):
        true_label = label_ind[l]
        true_number += np.sum(ClusterResult[label == true_label] == result_ind[l])
    accuracy.append(true_number / len(label))

score_result = pd.concat([candidate, pd.DataFrame(score), pd.DataFrame(accuracy)], axis = 1)
score_result.columns = ['gamma', 'NMI', 'Accuracy']
score_result

100%|██████████| 7/7 [04:39<00:00, 39.94s/it]


,gamma,NMI,Accuracy
0,0.01,0.222751,0.551913
1,0.05,0.195641,0.398907
2,0.10,0.156297,0.289617
3,0.50,0.203469,0.387978
4,1.00,0.197112,0.196721
5,5.00,0.149679,0.185792
6,10.00,0.153575,0.185792


In [23]:
score_result.iloc[np.where(score_result['NMI'] == max(score_result['NMI']))[0], :]

,gamma,NMI,Accuracy
0,0.01,0.222751,0.551913


In [24]:
score_result.iloc[np.where(score_result['Accuracy'] == max(score_result['Accuracy']))[0], :]

,gamma,NMI,Accuracy
0,0.01,0.222751,0.551913


In [ ]:
# ---------- Clustering on Graph ---------- #

In [25]:
ClusterResult = spectral_clustering(A, n_clusters = 5)

In [26]:
# NMI
normalized_mutual_info_score(label, ClusterResult)

0.0930857593978434

In [27]:
# Accuracy
label_count = pd.Series(label)
label_ind = label_count.value_counts().index

result_count = pd.Series(ClusterResult)
result_ind = result_count.value_counts().index

true_number = 0
for j in range(len(label_ind)):
    true_label = label_ind[j]
    true_number += np.sum(ClusterResult[label == true_label] == result_ind[j])
true_number / len(label)

0.37158469945355194